In [1]:
import pandas as pd
import spacy

from tqdm.notebook import tqdm
tqdm.pandas()



Ce code réalise une lemmatization à partir d'un modèle de machine learning issu de spacy.
installation du modèle : 
> python -m spacy download fr_core_news_sm

Ensuite on supprime certains mots fréquents, remis en forme et trouvés grâce à :
> from nltk.probability import FreqDist

Les nombres sont aussi filtrés.

J'ai réalisé cette approche car un  cleaning manuel était trop long (code en n² où n est le nombre de mots total du corpus). Le défaut de cette aproche est que potentiellement est elle trop bourrine et certains mots spécifiques qui nous intéressent ont disparu.

Il faut valider par une approche lexico et LDA les résultats => cf le code analyse_lexico.qmd

# Opening data

In [2]:
fichier = "../data/intermediate/base_merged.csv"

df = pd.read_csv(fichier, sep=';', quotechar='"')

df.sample(5)

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL
468,13,130098,Deux recensions d’ouvrages féministes récents ...,blog,TRADFEM,21/05/2023,Deux recensions d’ouvrages féministes récents ...,https://tradfem.wordpress.com/2023/05/21/deux-...
323,30,300004,Elle est présidente de l’American College of P...,Article d'opinion,NaN,2017-07-13 00:00:00,NaN,https://www.parti-de-la-france.fr/articles/Sel...
20,12,120021,"Le féminisme critique du genre, c’est quoi ?Le...",opinion,Dora Moutot & Marguerite Stern,18/10/2022,"Le féminisme critique du genre, c’est quoi ?",https://www.femelliste.com/articles-femellisme...
486,13,130116,Errements de l’adoption internationaleTRADFEM/...,blog,TRADFEM,10/05/2023,Errements de l’adoption internationale,https://tradfem.wordpress.com/2023/05/10/errem...
404,13,130034,CANADA : Quand le transactivisme devient une p...,blog,TRADFEM,26/08/2023,CANADA : Quand le transactivisme devient une p...,https://tradfem.wordpress.com/2023/08/26/canad...


# Cleaning and tokenisation

In [3]:
# Load the French language model
nlp = spacy.load("fr_core_news_sm")

# Define your list of words to exclude
words_to_filter = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent', 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos', 'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d", "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette', 'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–', "en", "autre", "ici", "ce", "ça", "cela", "ceci", "car", "ou", "où", "ni", "mais", "et", "donc", "parce", "que", "quand", "comment", "qui", "quoi",
    "à", "de", "pour", "sans", "sur","l","d","que","lui","n","s","c","j","me","alors","certain","chez","celer"
    "lequel","m","cas","tel","luire","après","souvent","aucun","toujours","également","jamais","hui","beaucoup","rien"
]

# Function to tokenize, filter, and lemmatize text, excluding numbers
def tokenize_filter_and_lemmatize(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Extract lemmatized forms of the words, excluding specific tokens and numbers
    lemmatized = " ".join([token.lemma_ for token in doc if token.lemma_ not in words_to_filter and not token.is_digit])
    return lemmatized

# Apply the function to the 'contenu' column
df['lemmatized_contenu'] = df['contenu'].progress_apply(tokenize_filter_and_lemmatize)

# Print the dataframe to see the result
df.sample(5)

  0%|          | 0/488 [00:00<?, ?it/s]

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL,lemmatized_contenu
42,17,170014,"Enfin, après tant d’années de luttes acharnées...",analyse,NaN,NaN,NaN,NaN,enfin tant d’ année lutte acharner l’ abolitio...
337,30,300018,Voici le début de mon dernier article sur « Ri...,Article d'opinion,NaN,2018-05-27 00:00:00,NaN,https://www.parti-de-la-france.fr/articles/Et-...,voici début dernier article Riposte Laïque viv...
125,20,200006,Les « soins d’affirmation du genre » sont un d...,analyse,Cry for Recognition,NaN,"GenderDissent – Transition your body, Destroy ...",NaN,soin d’ affirmation genre domaine médecine...
321,30,300002,La série télé quotidienne « Plus belle la vie ...,Article d'opinion,NaN,2018-03-08 00:00:00,NaN,https://www.parti-de-la-france.fr/articles/Pro...,série téler quotidien bel vie diffuser France ...
484,13,130114,Lu sur le substack du proféministe Graham Line...,blog,TRADFEM,11/05/2023,Lu sur le substack du proféministe Graham Linehan,https://tradfem.wordpress.com/2023/05/11/lu-su...,Lu substack proféministe Graham linehantradfem...


# Export

In [4]:
fichier = "../data/intermediate/base_lemmatized.csv"
df.to_csv(fichier, sep=';', index=False, quotechar='"')